In [ ]:
%%capture
!pip install transformers
!pip install evaluate
!pip install baal

In [ ]:
%%capture
!pip install transformers
!pip install evaluate
!pip install baal

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_theme()

In [ ]:
# set the seed: built-in python, numpy, and pytorch
seed = 13
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
device

In [ ]:
from transformers import AutoTokenizer, AutoModel,AutoModelForSequenceClassification

In [ ]:
# load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("nbroad/ESG-BERT")
model=AutoModelForSequenceClassification.from_pretrained("nbroad/ESG-BERT",num_labels=2,ignore_mismatched_sizes=True)

In [ ]:
text = []
with open('/content/climate-fever-dataset-r1.jsonl') as f:
    for jsonObj in f:
        t = json.loads(jsonObj)
        text.append(t)

In [ ]:
df=pd.DataFrame(columns=['claim','label'])
for i in text:
  row={'claim':i['claim'],'label':i['claim_label']}
  df=df.append(row,ignore_index=True)

df['label']=df['label'].apply(lambda x: 1 if x=='SUPPORTS' else 0)

In [ ]:
unlabeled_index=random.sample(range(0,1534),1380)
labeled_index=list(set(range(1534))-set(unlabeled_index))


In [ ]:
train_tokenized=tokenizer(all_data['claim'].to_list(), truncation=True,max_length=330,padding=True,return_tensors='pt')

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        #item = {'text':self.text,'label':self.labels}
        item['label'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
#raw_train_set=Dataset(all_data['claim'].to_list(),all_data['label'].to_list())
raw_train_set=Dataset(train_tokenized,all_data['label'].to_list())

In [ ]:
active_set = ActiveLearningDataset(raw_train_set)
active_set.can_label = False # Need to manually do this for research

# lets randomly label 100 samples, therefore len(active_set) should be 100
active_set.label_randomly(150)
assert len(active_set) == 150
print(len(active_set.pool))

In [ ]:
active_set.n_labelled

In [ ]:
active_set.n_unlabelled


In [ ]:
use_cuda = torch.cuda.is_available()

model = patch_module(model)
if use_cuda:
    model.cuda()
init_weights = deepcopy(model.state_dict())

In [ ]:
from baal.active import get_heuristic

heuristic = get_heuristic('bald')

In [ ]:
from transformers import TrainingArguments
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active.active_loop import ActiveLearningLoop

In [ ]:
training_args = TrainingArguments(
    output_dir='outputs',  # output directory
    num_train_epochs=4,  # total # of training epochs per AL step
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,  # batch size for evaluation
    #weight_decay=0.01,  # strength of weight decay
    logging_dir='logs',  # directory for storing logs
    learning_rate=1e-4, #0.00012922855,
    overwrite_output_dir=True,
    )

# create the trainer through Baal Wrapper
baal_trainer = BaalTransformersTrainer(model=model,
                                       args=training_args,
                                       train_dataset=active_set,
                                       tokenizer=None)


active_loop = ActiveLearningLoop(active_set,
                                 baal_trainer.predict_on_dataset,
                                 heuristic, 10, iterations=3)

for epoch in range(2):
    baal_trainer.train()
    
    should_continue = active_loop.step()

    # We reset the model weights to relearn from the new train set.
    baal_trainer.load_state_dict(init_weights)
    baal_trainer.lr_scheduler = None
    if not should_continue:
        break

# at each Active step we add 10 samples to labelled data. At this point we should have 30 samples added
# to the labelled part of training set.
print(len(active_set))